## **Setting Up Fabric Integration (Tool) with Azure AI Agents**

### **📝 Overview**

Integrate your Azure AI Agent with Fabric `Data agent` to unlock powerful data analysis capabilities. Fabric `Data Agent` transforms enterprise data into conversational Q&A systems, allowing users to interact with data through chat and uncover actionable insights effortlessly.

When a user sends a query, the Azure AI Agent first determines if Fabric `Data agent` should be leveraged. If so, it uses the end user’s identity to generate queries over accessible data, and then returns responses based on the queried results. With **on-behalf-of (OBO) authorization**, this integration simplifies secure access to enterprise data in Fabric while ensuring robust protection and proper access control.

### **✅ Prerequisites**

- **Published Fabric AI Skill**: Ensure you have a published Fabric `Data Agent`.
- **Permission/Role Assignment**:
  - **Access to AI Skill**: Users must have at least “Read” access to the `Data Agent` and connected data sources.
  - **RBAC for Foundry Project**: End users must have the `AI Developer` role assigned.

### **Step-by-Step Guide**

1. **Access the Agent Playground**:
   - Navigate to the Agent Playground in your Azure AI environment.

2. **Create or Use an Existing Agent**:
   - Either create a new agent or select an existing one to configure.

3. **Add a Knowledge Source**:
   - Click to add a knowledge source and select **Microsoft Fabric**.
   - If you don’t see this option, ensure the feature flag `&flight=MicrosoftFabricKnowledge` is enabled.

4. **Create a New Connection**:
   - Provide the following key-value pairs for the connection:
     - `workspace-id`: `xxx`
     - `artifact-id`: `xxx`
     - `audience`: `xxx`
   - These values can be found in your Data Agent endpoint. The endpoint format is:
     ```
     https://daily.powerbi.com/groups/<workspace_id>/aiskills/<artifact-id>
     ```

5. **Mark Fields as Secret**:
   - Check the “is secret” option for all fields to ensure secure handling of sensitive information.

6. **Name Your Connection**:
   - Assign a meaningful name to your connection for easy identification.

7. **Start Chatting with Fabric**:
   - Once the connection is set up, you can begin interacting with Fabric through your Azure AI Agent.


In [1]:
import os

from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Define the target directory
target_directory = os.getcwd()  # Get the current working directory

# Move one directory back
parent_directory = os.path.dirname(target_directory)

# Check if the parent directory exists
if os.path.exists(parent_directory):
    # Change the current working directory to the parent directory
    os.chdir(parent_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Parent directory {parent_directory} does not exist.")

Directory changed to c:\Users\pablosal\Desktop\gbb-ai-agenticrag


In [2]:
import importlib.metadata as md

# Versions - we are currently 1.0.0b9 of azure-ai-projects
print("semantic-kernel version:", md.version("semantic-kernel"))
# if you want to Upgrade the SDKs, uncomment the line below but code might break
# %pip install -U semantic-kernel azure-ai-projects azure-identity

semantic-kernel version: 1.28.1


In [3]:
import os
import re
import time
import logging
import json
from datetime import datetime as pydatetime
from typing import Any, List, Dict, Optional
from dotenv import load_dotenv
import asyncio
from datetime import timedelta

# Azure AI Projects
from azure.identity.aio import DefaultAzureCredential
from azure.core.exceptions import HttpResponseError

# semantic kernel
from semantic_kernel.contents import AuthorRole
from semantic_kernel.agents.azure_ai.azure_ai_agent import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.open_ai.run_polling_options import RunPollingOptions

# Load environment variables from .env file
load_dotenv()

# configure logging
from utils.ml_logging import get_logger

logger = get_logger()

In [ ]:
from azure.core.exceptions import ServiceRequestError
from azure.ai.projects.aio import AIProjectClient

async def get_connection_id(client: AIProjectClient, env_var: str) -> Optional[str]:
    """
    Retrieves the connection object using a connection name stored in an environment variable.

    Args:
        client: The Azure AI Project client.
        env_var (str): The environment variable holding the connection name.

    Returns:
        Connection object if found, otherwise raises an error.
    """
    connection_name = os.getenv(env_var)
    if not connection_name:
        logger.error(f"Missing environment variable: '{env_var}'")
        raise ValueError(f"Environment variable '{env_var}' is required.")

    try:
        connection = await client.connections.get(connection_name=connection_name)
        logger.info(f"Retrieved Connection ID for {env_var}: {connection.id}")
        return connection
    except Exception as e:
        logger.error(f"Failed to retrieve connection for {env_var}: {e}")
        raise

import json
import logging
from datetime import timedelta

def process_citations(content) -> str:
    """
    Return content plus any citations in Markdown format, ensuring no duplicated citations.

    Args:
        content: The content object retrieved from the agent.

    Returns:
        A string containing the agent's text plus any unique citations in Markdown format.
    """
    combined_content = content.content

    if hasattr(content, "items"):
        unique_citation_set = set()
        for item in content.items:
            if item.content_type == "annotation" and item.url:
                unique_citation_set.add((item.quote, item.url))

        if unique_citation_set:
            combined_content += "\n\n**Citations:**\n"
            for quote, url in unique_citation_set:
                combined_content += f"- **Quote**: {quote}  \n"
                combined_content += f"  **URL**: [{url}]({url})\n"

    return combined_content

async def run_agent(
    project_client,
    agent_id: str,
    user_input: str
) -> str:
    """
    Runs the data-retrieval agent to process a single user input and returns
    a single string containing both user input and the agent responses.

    Args:
        project_client: A client object for interacting with Azure AI project resources.
        agent_id: The ID of the agent to retrieve from project_client.
        user_input: A string representing the user's query.

    Returns:
        A single string with user input and each agent response (enriched with citations).
    """
    # Initialize the string to include the user query
    responses = f"👤 User: {user_input}\n"

    # Fetch the agent definition using the provided ID
    agent_definition = await project_client.agents.get_agent(agent_id)
    
    # Create the AzureAIAgent instance
    agent = AzureAIAgent(
        client=project_client,
        definition=agent_definition,
        polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Create a new conversation thread
    thread = await project_client.agents.create_thread()

    try:
        # Send user input to the agent
        await agent.add_chat_message(thread_id=thread.id, message=user_input)
        
        # Stream non-tool responses from the agent
        async for content in agent.invoke(thread_id=thread.id):
            enriched_content = process_citations(content)
            responses += f"\n🤖 {agent.name}: {enriched_content}"

    except HttpResponseError as e:
        try:
            error_json = json.loads(e.response.content)
            message = error_json.get("Message", "No error message found")
            logging.error(f"❌ **HTTP Error:** {message}")
            responses += f"\n❌ **HTTP Error:** {message}"
        except json.JSONDecodeError:
            logging.error(f"❌ **Non-JSON Error Content:** {e.response.content}")
            responses += f"\n❌ **Non-JSON Error Content:** {e.response.content}"

    return responses, thread.id

### **Create Client and Load Azure AI Foundry**

Here, we initialize the Azure AI client using DefaultAzureCredential. This allows us to authenticate and connect to the Azure AI service.

In [5]:
project_client = AzureAIAgent.create_client(credential=DefaultAzureCredential(),
conn_str=os.getenv("AZURE_AI_FOUNDRY_CONNECTION_STRING"))

In [6]:
from azure.ai.projects.models import (
    FabricTool,
    ToolSet,
)

# Initialize Azure AI Agent settings
dataretrievalagent_settings = AzureAIAgentSettings.create()

toolset = ToolSet()

try:
    # Retrieve and add Fabric Tool
    fabric_connection = await get_connection_id(project_client, "TOOL_CONNECTION_NAME_FABRIC")
    toolset.add(FabricTool(connection_id=fabric_connection.id))

    logger.info("Successfully created ToolSet with SharePoint and Fabric tools.")
except Exception as e:
    logger.error(f"Failed to create ToolSet: {e}")
    raise

The create method is deprecated. Use the __new__ method instead.
2025-04-30 02:20:33,231 - micro - MainProcess - ERROR    Failed to retrieve connection for TOOL_CONNECTION_NAME_FABRIC: (UserError) Connection glucose-fabric-connection can't be found in this workspace
Code: UserError
Message: Connection glucose-fabric-connection can't be found in this workspace (2375541312.py:get_connection_id:25)
2025-04-30 02:20:33,231 - micro - MainProcess - ERROR    Failed to create ToolSet: (UserError) Connection glucose-fabric-connection can't be found in this workspace
Code: UserError
Message: Connection glucose-fabric-connection can't be found in this workspace (2316211666.py:<module>:18)


ResourceNotFoundError: (UserError) Connection glucose-fabric-connection can't be found in this workspace
Code: UserError
Message: Connection glucose-fabric-connection can't be found in this workspace

In [ ]:
dataretrievalagent_settings_definition = await project_client.agents.create_agent(
    model=dataretrievalagent_settings.model_deployment_name,
    name="FabricDataRetrievalAgent",
    description=(
        "An AI agent specialized in retrieving and integrating structured data from Microsoft Fabric. "
        "This includes performance metrics, experiment results, and analytics data. "
        "The agent is designed to assist in research and development by providing accurate, relevant, and actionable data. "
        "If no relevant data is found, the agent must clearly indicate this and provide suggestions for alternative queries or data sources."
    ),
    instructions=(
        "### Role & Objective\n"
        "You are a research-focused AI assistant responsible for retrieving structured data exclusively from Microsoft Fabric. "
        "Your goal is to provide precise, well-referenced, and relevant responses to support research and development efforts.\n\n"
        
        "### Data Retrieval & Prioritization\n"
        "1. **Structured Data (Microsoft Fabric):** \n"
        "   - Retrieve data from Microsoft Fabric when the query involves numerical metrics, performance statistics, or structured analytics.\n"
        "   - Data is available to support comparisons of the accuracy and reliability of two glucose monitoring products (Product A and Product B). "
        "This structured data evaluates performance across different glucose ranges and includes MARD percentages, accuracy within ±20 mg/dL/±20%, and the number of readings for each product.\n"
        "   - Example: clinical glucose monitoring studies between Product A or Product B.\n\n"
        
        "2. **Integrated Queries:** \n"
        "   - If the query requires multiple structured metrics, retrieve and integrate the results for a comprehensive response.\n"
        "   - Ensure clarity in presenting combined insights.\n\n"
        
        "3. **Fallback Behavior:** \n"
        "   - If no relevant data is found in Microsoft Fabric, respond with:\n"
        "     - A clear statement that no relevant data was found.\n"
        "     - Suggestions for alternative queries or data sources (if applicable).\n"
        "     - Example: 'No relevant data was found for the requested query in Microsoft Fabric. Consider refining your query or exploring other data sources.'\n\n"
        
        "### Response Quality\n"
        "1. **Accuracy & Relevance:** Always prioritize retrieving the most current and applicable data from Microsoft Fabric.\n"
        "2. **Clarity & Transparency:** Clearly indicate the data source(s) used and any limitations in the available information.\n"
        "3. **Fallback Handling:** If no relevant data is found, provide a professional and helpful fallback response as outlined above.\n"
        "4. **Professionalism:** Present findings in a structured and concise manner to facilitate decision-making.\n"
    ),
    toolset=toolset,
    headers={"x-ms-enable-preview": "true"},
    temperature=0.7,
    top_p=1,
    metadata={
        "use_case": "Structured Data Retrieval for R&D",
        "data_source": "Microsoft Fabric (structured metrics only)",
    },
)

# Print the agent's run ID (agent ID)
print(f"DataRetrievalAgent Run ID: {dataretrievalagent_settings_definition.id}")

In [ ]:
user_query = "In which glucose ranges does Product A underperform compared to Product B, and what clinical impact could this have?"
fabric_response, threadID = await run_agent(project_client, "asst_iIMaKxWz4JUJ5YyWqoSR9t3n", user_query)